In [24]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import torchvision.models as models
import torch.nn as nn
import torch

In [25]:
IMAGE_SIZE = 224
BATCH_SIZE = 16
NUM_WORKERS = 2

In [26]:
test_transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225]),
])

dataset_path = '../../../dataset/test/'

test_dataset = datasets.ImageFolder(dataset_path, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [27]:
num_classes = len(test_dataset.classes)

model = models.densenet169(weights=models.DenseNet169_Weights.IMAGENET1K_V1)
model.classifier = nn.Sequential(nn.Dropout(0.3), nn.Linear(model.classifier.in_features, num_classes))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

trained_model_path = 'best_densenet169.pth'
model.load_state_dict(torch.load(trained_model_path, map_location=device))

criterion = nn.CrossEntropyLoss()

C:\Users\Raiza Rahman\AppData\Local\Temp\ipykernel_6640\903797191.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(trained_model_path, m

In [28]:
def test_model(model, device, loader, criterion):
    model.eval()
    test_loss, correct, total = 0.0, 0, 0
    
    with torch.no_grad():
        for images, labels, in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)
            
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    avg_loss = test_loss/total
    accuracy = correct/total
    
    return avg_loss, accuracy

In [29]:
test_loss, test_acc = test_model(model, device, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

Test Loss: 0.2904, Test Accuracy: 0.9015


In [30]:
correct = 0
total = 0

all_preds = []
all_labels = []

model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = 100 * correct/total
print(f"Test Accuracy: {accuracy:.2f}%")

report = classification_report(all_preds, all_labels, target_names=test_dataset.classes, output_dict=True)

print(f"{'Class':<25} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'Accuracy':<10}")
print("-" * 65)

for class_name in test_dataset.classes:
    cls_report = report[class_name]
    precision = cls_report['precision']
    recall = cls_report['recall']
    f1 = cls_report['f1-score']
    support = cls_report['support']
    acc = (recall * support) / support

    print(f"{class_name:<25} {precision:<10.2f} {recall:<10.2f} {f1:<10.2f} {acc*100:<10.2f}")

Test Accuracy: 90.15%
Class                     Precision  Recall     F1-Score   Accuracy  
-----------------------------------------------------------------
algal_spot                0.99       0.87       0.93       87.05     
brown_blight              0.91       0.82       0.87       82.43     
gray_blight               0.74       0.89       0.81       88.97     
healthy                   0.94       0.90       0.92       90.38     
helopeltis                0.93       0.93       0.93       93.29     
red-rust                  0.83       0.91       0.87       90.91     
red-spider-infested       0.95       1.00       0.98       100.00    
red_spot                  0.91       0.97       0.94       96.89     
white-spot                0.91       1.00       0.95       100.00    
